In [1]:
import tensorflow as tf
import tensorflow_addons as tfa

In [12]:
y_true = tf.constant([[1.0], [1.0], [1.0], [0.0]], dtype=tf.float32)

y_pred = tf.constant([[.8], [0.2], [.7], [.7]], dtype=tf.float32)
y_pred_round = tf.constant([[1.0], [0.0], [1.0], [1.0]], dtype=tf.float32)

In [13]:
mcc = tfa.metrics.MatthewsCorrelationCoefficient(num_classes=1)

In [9]:
f1 = tfa.metrics.F1Score(num_classes=1)

In [14]:
mcc.update_state(y_true, y_pred_round)

In [11]:
mcc.result()

<tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.6123724], dtype=float32)>

In [15]:
mcc.result()

<tf.Tensor: shape=(1,), dtype=float32, numpy=array([-0.33333334], dtype=float32)>